In [1]:
import pandas as pd
import numpy as np
#from google.colab import drive
from statsmodels.tools.eval_measures import rmse

In [2]:
#drive.mount('/content/drive')

In [3]:
def df_split(df):
  categoria = []
  for x in df['category_name']:
    categoria.append(x.split('/'))
  return categoria


def sub_categoria(df):  
  lista = df_split(df)
  for y in range(len(lista)):
    lista[y] = lista[y][1]
  return lista

In [4]:
filtro_home = pd.read_csv('https://github.com/AugustoCRX/bootcamp_blue/blob/main/novos_arquivos_treino/Limpeza%20v3/filtro_Home_train_v3.csv?raw=true')

In [5]:
filtro_home.head()

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock,date2,day,month,year,description_clear,name_clear,brands_filter
0,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,Other,35.0,1,New with tags. Leather horses. Retail for [rm]...,19-2-2018,1,2018-02-19,19,2,2018,new tag leather hors retail rm stand foot high...,leather hors statu,upper_pop_brand
1,"Rae Dunn ""HUGS"" Mug",1,Home/Kitchen & Dining/Coffee & Tea Accessories,Rae Dunn,21.0,0,"New 2017 Rae Dunn Holiday ""HUGS"" mug. Another ...",29-6-2018,17,2018-06-29,29,6,2018,new 2017 rae dunn holiday hug mug anoth great ...,rae dunn `` hug '' mug,rae dunn
2,Listing for Aknuckles1,1,Home/Home Décor/Home Fragrance,Nostalgia Electrics,35.0,0,WARMERS PICTURED & 6 SAMPLE PACKAGES OF HIGHLY...,6-4-2018,1,2018-06-04,4,6,2018,warmer pictur 6 sampl packag highli scent wax ...,list aknuckles1,nostalgia
3,Rae Dunn Beauty and the Beast,1,Home/Kitchen & Dining/Coffee & Tea Accessories,Rae Dunn,26.0,0,Brand new! No cracks or chips! I package every...,7-2-2018,7,2018-07-02,2,7,2018,brand new crack chip packag everyth extra care...,rae dunn beauti beast,rae dunn
4,Aura Fluorite Double Terminated Wand,1,Home/Home Décor/Other,Other,19.0,1,"Aura Fluorite: ⋅ The ""protection stone"" ⋅ Grou...",21-6-2018,13,2018-06-21,21,6,2018,aura fluorit protect stone ground spiritu intu...,aura fluorit doubl termin wand,popular_brand


In [6]:
filtro_home['sub_category_name'] = sub_categoria(filtro_home)

In [7]:
filtro_home.sub_category_name.value_counts()

Home Décor                15699
Kitchen & Dining          14982
Seasonal Décor             3576
Bedding                    3301
Storage & Organization     2167
Home Appliances            2029
Cleaning Supplies          1809
Artwork                     955
Bath                        921
Kids' Home Store            537
Other                       389
Furniture                   132
Name: sub_category_name, dtype: int64

# modelo 01

Regressão Linear sem restrição de preço e com variaveis dummies nas variaveis categoricas sub_category_name, item_condition_id, shipping e brands_filter

In [8]:
import statsmodels.api as sm

In [9]:
filtro_home = pd.get_dummies(filtro_home, columns=['sub_category_name','brands_filter','item_condition_id','shipping'])

In [10]:
X = filtro_home.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y = filtro_home.price
X = sm.add_constant(X)
m1_rl = sm.OLS(exog=X,endog=y)
m1_result = m1_rl.fit()
print(m1_result.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     710.5
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        15:43:19   Log-Likelihood:            -2.0373e+05
No. Observations:               46497   AIC:                         4.076e+05
Df Residuals:                   46441   BIC:                         4.081e+05
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [11]:
filtro_home['price'].describe()

count    46497.000000
mean        25.965471
std         26.256653
min          3.000000
25%         12.000000
50%         19.000000
75%         31.000000
max        848.000000
Name: price, dtype: float64

In [12]:
m1_ypred = m1_result.predict(X)
m1_RMSE = rmse(y,m1_ypred)
m1_RMSE.round(2)

19.35

# Modelo 02
Regressão Linear com restrição de preço ate 150 dolares e com variaveis dummies nas variaveis categoricas sub_category_name e brands_filter




In [13]:
filtro_home_ajustado = pd.read_csv('https://github.com/AugustoCRX/bootcamp_blue/blob/main/novos_arquivos_treino/Limpeza%20v3/filtro_Home_train_v3.csv?raw=true')
filtro_home_ajustado['sub_category_name'] = sub_categoria(filtro_home_ajustado)
filtro_home_ajustado = pd.get_dummies(filtro_home_ajustado, columns=['sub_category_name','brands_filter'])

In [14]:
filtro_home_ajustado = filtro_home_ajustado[filtro_home_ajustado['price'] <= 150]

In [15]:
X1 = filtro_home_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y1 = filtro_home_ajustado.price
X1 = sm.add_constant(X1)
m2_rl = sm.OLS(exog=X1,endog=y1)
m2_result = m2_rl.fit()
print(m2_result.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.537
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     1048.
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        15:43:21   Log-Likelihood:            -1.8509e+05
No. Observations:               46152   AIC:                         3.703e+05
Df Residuals:                   46100   BIC:                         3.707e+05
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [16]:
filtro_home_ajustado['price'].describe()

count    46152.000000
mean        24.556715
std         19.619756
min          3.000000
25%         12.000000
50%         19.000000
75%         31.000000
max        150.000000
Name: price, dtype: float64

In [17]:
m2_ypred = m2_result.predict(X1)
m2_RMSE = rmse(y1,m2_ypred)
m2_RMSE.round(2)

13.35

# Modelo 03

Arvore de Regressão usando como criterio de seleção dos nós a métrica R²

In [18]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [19]:
X2 = filtro_home.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y2 = filtro_home.price
m3_dtr = DecisionTreeRegressor(random_state=10)
result2 = m3_dtr.fit(X2,y2)
m3_r2 = result2.score(X2,y2)
m3_ypred = result2.predict(X2)
m3_mse = mean_squared_error(y2,m3_ypred)
m3_rmse = m3_mse**0.5
print(f"R² = {m3_r2.round(2)}")
print(f"MSE = {m3_mse.round(2)}")
print(f"RMSE = {m3_rmse.round(2)}")

R² = 0.5
MSE = 344.56
RMSE = 18.56


# Modelo 04

Arvore de Regressão com restrição de preço ate 150 dolares e com variaveis dummies nas variaveis categoricas sub_category_name e brands_filter

In [20]:
X3 = filtro_home_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y3 = filtro_home_ajustado.price
m4_dtr = DecisionTreeRegressor(random_state=10)
result3 = m4_dtr.fit(X3,y3)
m4_r2 = result3.score(X3,y3)
m4_ypred = result3.predict(X3)
m4_mse = mean_squared_error(y3,m4_ypred)
m4_rmse = m4_mse**0.5
print(f"R² = {m4_r2.round(2)}")
print(f"MSE = {m4_mse.round(2)}")
print(f"RMSE = {m4_rmse.round(2)}")

R² = 0.56
MSE = 169.07
RMSE = 13.0


# Resultados

| Modelo | R² |RMSE | Observação |
| --- | --- | --- | --- |
| Modelo 01 | 45,7% | 19,35 | Regressão Linear |
| Modelo 02 | 53,7% | 13,35 | Regressão Linear (preço <= 150) |
| Modelo 03 | 50,0% | 18,56 | Arvore Regressão (criterio = R²) |
| Modelo 04 | 56,0% | 13,00 | Arvore Regressão (criterio = R², preço <= 150) |